# Imports

In [1]:
import pandas as pd
import numpy as np

In [27]:
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"

In [28]:
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

# Explore Data

In [4]:
basics.info()
basics.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9446373 entries, 0 to 9446372
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 648.6+ MB


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [5]:
akas.info()
akas.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34190214 entries, 0 to 34190213
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 2.0+ GB


,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [6]:
ratings.info()
ratings.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1257088 entries, 0 to 1257087
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1257088 non-null  object 
 1   averageRating  1257088 non-null  float64
 2   numVotes       1257088 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 28.8+ MB


,tconst,averageRating,numVotes
0,tt0000001,5.7,1925
1,tt0000002,5.8,261
2,tt0000003,6.5,1741
3,tt0000004,5.6,176
4,tt0000005,6.2,2554


# Preprocessing

## Replace "\N" with np.nan

In [7]:
basics.isna().sum()

tconst             0
titleType          0
primaryTitle      11
originalTitle     11
isAdult            0
startYear          0
endYear            0
runtimeMinutes     0
genres            10
dtype: int64

In [29]:
basics.replace({'\\N':np.nan}, inplace=True)

In [30]:
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle           11
originalTitle          11
isAdult                 1
startYear         1265632
endYear           9346581
runtimeMinutes    6756650
genres             431735
dtype: int64

In [31]:
akas.replace({'\\N':np.nan}, inplace=True)

In [32]:
ratings.replace({'\\N':np.nan}, inplace=True)

## Exclude any movie with missing values for genre or runtime

In [33]:
basics = basics[(basics['runtimeMinutes'].notna())]

In [34]:
basics = basics[(basics['genres'].notna())]

## Include only full-length movies (titleType = "movie")

In [35]:
basics = basics[(basics['titleType']=='movie')]

## Include only movies that were released 2000 - 2021 (include 2000 and 2021)

In [38]:
basics['startYear'] = basics['startYear'].astype(float)
basics['endYear'] = basics['endYear'].astype(float)

In [44]:
basics = basics[(basics['startYear']>=2000) & (basics['startYear']<=2022)]

## Include only fictional movies (not from documentary genre)

In [45]:
# Exclude movies that are included in the documentary category.
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]

## Include only movies that were released in the United States

In [46]:
akas = akas[(akas['region']=='US')]

In [47]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics['tconst'].isin(akas['titleId'])
keepers

34804       True
61117       True
67670       True
77965      False
86802       True
           ...  
9446045     True
9446054     True
9446093    False
9446138     True
9446222    False
Name: tconst, Length: 145632, dtype: bool

In [48]:
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34804,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61117,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020.0,NaN,70,Drama
67670,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86802,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
93939,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama
...,...,...,...,...,...,...,...,...,...
9445509,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019.0,NaN,74,Drama
9445905,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019.0,NaN,97,"Comedy,Drama,Fantasy"
9446045,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama
9446054,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"


In [49]:
ratings_keepers =ratings['tconst'].isin(akas['titleId'])
ratings_keepers

0           True
1           True
2          False
3          False
4           True
           ...  
1257365    False
1257366     True
1257367    False
1257368    False
1257369    False
Name: tconst, Length: 1257370, dtype: bool

In [50]:
ratings = ratings[ratings_keepers]
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1926
1,tt0000002,5.8,261
4,tt0000005,6.2,2555
5,tt0000006,5.1,175
6,tt0000007,5.4,797
...,...,...,...
1257344,tt9916204,8.2,250
1257350,tt9916348,8.5,17
1257351,tt9916362,6.4,5053
1257355,tt9916428,3.8,14


# Final Summary after Filtering

In [51]:
basics.info()
basics.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84004 entries, 34804 to 9446138
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          84004 non-null  object 
 1   titleType       84004 non-null  object 
 2   primaryTitle    84004 non-null  object 
 3   originalTitle   84004 non-null  object 
 4   isAdult         84004 non-null  object 
 5   startYear       84004 non-null  float64
 6   endYear         0 non-null      float64
 7   runtimeMinutes  84004 non-null  object 
 8   genres          84004 non-null  object 
dtypes: float64(2), object(7)
memory usage: 6.4+ MB


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34804,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61117,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020.0,NaN,70,Drama
67670,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86802,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
93939,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [56]:
akas.info()
akas.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1392212 entries, 5 to 34183296
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1392212 non-null  object
 1   ordering         1392212 non-null  int64 
 2   title            1392212 non-null  object
 3   region           1392212 non-null  object
 4   language         3738 non-null     object
 5   types            967379 non-null   object
 6   attributes       45511 non-null    object
 7   isOriginalTitle  1390867 non-null  object
dtypes: int64(1), object(7)
memory usage: 95.6+ MB


,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [57]:
ratings.info()
ratings.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 478814 entries, 0 to 1257366
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         478814 non-null  object 
 1   averageRating  478814 non-null  float64
 2   numVotes       478814 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 14.6+ MB


,tconst,averageRating,numVotes
0,tt0000001,5.7,1926
1,tt0000002,5.8,261
4,tt0000005,6.2,2555
5,tt0000006,5.1,175
6,tt0000007,5.4,797


# Downloading Data into File

In [54]:
# example making new folder with os
import os
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

[]

In [58]:
## Save current dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

In [59]:
# Open saved file and preview again
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama
